In [5]:
import argparse
import pickle as pkl
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
# from clickstream_experiment.source.clickstream import ClickStream
from gensim.models import KeyedVectors
from gensim.models.word2vec import LineSentence
# from FlowHMM.model.discretized_HMM import DiscreteHMM

In [6]:
DATA_SET = 'train'
PROJECT_PATH = '/Ziob/kabalce/FlowHmm'
w2v_dim = 100


In [9]:
vectors = KeyedVectors.load(f'{PROJECT_PATH}/data/preprocessed_data/vectors_train_100.kv')

In [14]:
vectors.word_vec()

/tmp/ipykernel_1968135/2997131487.py:1: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  vectors.word_vec()


TypeError: KeyedVectors.get_vector() missing 1 required positional argument: 'key'

In [21]:
vectors.key_to_index

AttributeError: The index2entity attribute has been replaced by index_to_key since Gensim 4.0.0.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [28]:
import numpy as np
np.concatenate([vectors.get_vector(k).reshape(1, -1) for k in list(vectors.key_to_index.keys())])

array([[ 0.37812564,  0.9615767 ,  0.9456675 , ..., -1.8996654 ,
         1.8654928 , -0.6869288 ],
       [-0.0635553 ,  2.4719412 ,  0.12076852, ...,  0.5209727 ,
         0.2821499 , -0.03281005],
       [-0.12845026, -0.72366726, -2.040437  , ..., -0.00900516,
         2.5470822 ,  0.42293575],
       ...,
       [-0.03972329,  0.00678696, -0.02461287, ..., -0.02534718,
         0.08919474, -0.0304991 ],
       [-0.06562883,  0.05732198, -0.02860171, ..., -0.00453127,
         0.01211101, -0.10140423],
       [-0.02814574,  0.04212878, -0.04877787, ..., -0.01338863,
        -0.02220276, -0.04998498]], dtype=float32)

In [35]:
import pickle as pkl

with open("/ziob/klaudia/FlowHMM/clickstream_experiment/data/preprocessed_data/train_test_data_20_5_5_2023-04-24 10:28:26.059415.pkl", 'rb') as f:
    data = pkl.load(f)

In [36]:
data.keys()

dict_keys(['Xd_train', 'Xd_test', 'Xc_train', 'Xc_test', 'lengths_train', 'lengths_sub_train', 'lengths_test', 'myHMM.nodes'])

In [37]:
from hmmlearn.hmm import GaussianHMM

In [40]:
standardHMM = GaussianHMM(n_components=5, n_iter=150, verbose=True)
standardHMM.fit(data['Xc_train'], data['lengths_sub_train'])

print(
    f"Mean loglikelihood from standard implementation on test set: {standardHMM.score(data['Xc_test'], data['lengths_test']) / data['Xc_test'].shape[0]}"
)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
         1   -52313874.2868             +nan
         2   -51113598.8539    +1200275.4329
         3   -50634967.2154     +478631.6384
         4   -49998409.5945     +636557.6209
         5   -49422487.5993     +575921.9952
         6   -49048329.9501     +374157.6493
         7   -48826694.5717     +221635.3783
         8   -48685856.9812     +140837.5906
         9   -48576905.7738     +108951.2073
        10   -48468382.6066     +108523.1673
        11   -48336860.7382     +131521.8684
        12   -48185178.0928     +151682.6454
        13   -48076665.9299     +108512.1630
        14   -48015575.2244      +61090.7055
        15   -47965837.1083      +49738.1161
        16   -47903315.0877      +62522.0205
        17   -478126

Mean loglikelihood from standard implementation on test set: -35.54766043372142


In [55]:
lens = np.concatenate([np.array([0]), np.cumsum(data['lengths_test'])])
acc = 0
for i in range(lens.shape[0] - 1):
    states = standardHMM.predict(data['Xc_test'][lens[i]:lens[i+1]])
    print(states)
    s_pred = standardHMM.predict(data['Xc_test'][lens[i]:(lens[i+1] - 1)])
    print(s_pred)
    state_pred = np.argmax(standardHMM.transmat_[s_pred[-1]])
    acc += states[-1] == state_pred

print(acc / (lens.shape[0] - 1))

[2 0 0 0 0 2 0 0 0 0 0 0 2 2 4 2 0 0 0 0 0 0 0 2 0 0 0 0 0]
[2 0 0 0 0 2 0 0 0 0 0 0 2 2 4 2 0 0 0 0 0 0 0 2 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 4 0 0 0 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0 4]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 4 0 0 0 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
[3 3 2 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 2 2 4 3 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 3 3 2 2 2 2]
[3 3 2 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 2 2 4 3 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 3 3 2 2 2]
[4 2 0 2 2 2 4 2 4 2 4 2 2]
[4 2 0 2 2 2 4 2 4 2 4 2]
[1 1 1 1 1 1 1 1]
[1 1 1 1 1 3 3]
[3 1 1 1 1 1 1 1 1 1 4 2 1 1 1 1 1 2 3 3 3 2 2 2]
[3 1 1 1 1 1 1 1 1 1 4 2 1 1 1 1 1 2 3 3 3 2 2]
[3 3 3 1 1 1 1]
[3 3 3 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [53]:
standardHMM.transmat_

array([[0.87035161, 0.01872789, 0.077381  , 0.00708183, 0.02645767],
       [0.02419319, 0.84996712, 0.02514759, 0.09068947, 0.01000262],
       [0.16230877, 0.05363991, 0.42512403, 0.23504158, 0.1238857 ],
       [0.0068478 , 0.06950203, 0.11283515, 0.77033255, 0.04048247],
       [0.12370543, 0.05065617, 0.2296121 , 0.18308997, 0.41293633]])

In [56]:
standardHMM_20 = GaussianHMM(n_components=20, n_iter=40, verbose=True)
standardHMM_20.fit(data['Xc_train'], data['lengths_sub_train'])

print(
    f"Mean loglikelihood from standard implementation on test set: {standardHMM_20.score(data['Xc_test'], data['lengths_test']) / data['Xc_test'].shape[0]}"
)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
         1   -52398024.7137             +nan
         2   -49638713.7813    +2759310.9325
         3   -48702084.6491     +936629.1322
         4   -47428883.0563    +1273201.5928
         5   -46690117.3622     +738765.6941
         6   -46358499.9145     +331617.4476
         7   -46060201.6774     +298298.2371
         8   -45763587.7669     +296613.9105
         9   -45603960.3898     +159627.3772
        10   -45508558.5894      +95401.8003
        11   -45436873.9265      +71684.6629
        12   -45374900.3608      +61973.5658
        13   -45329530.3017      +45370.0591
        14   -45296727.3007      +32803.0010
        15   -45265755.5720      +30971.7286
        16   -45234534.5770      +31220.9950
        17   -452050

KeyboardInterrupt: 

In [57]:
print(
    f"Mean loglikelihood from standard implementation on test set: {standardHMM_20.score(data['Xc_test'], data['lengths_test']) / data['Xc_test'].shape[0]}"
)

Mean loglikelihood from standard implementation on test set: -33.8278725325801


In [66]:
lens_20 = np.concatenate([np.array([0]), np.cumsum(data['lengths_test'])])
acc_20 = 0
for i in range(lens_20.shape[0] - 1):
    states = standardHMM_20.predict(data['Xc_test'][lens_20[i]:lens_20[i+1]])
    print(states)
    s_pred = standardHMM_20.predict(data['Xc_test'][lens_20[i]:(lens_20[i+1] - 1)])
    print(s_pred)
    state_pred = np.argmax(standardHMM_20.transmat_[s_pred[-1]])
    acc_20 += states[-1] == state_pred

print(acc_20 / (lens_20.shape[0] - 1))

[16 14 18 18 18 16 18 18 18 18 18 18 16 16  0 16 18 18 18 18 18 18 18 16
  4 14 14 18 18]
[16 14 18 18 18 16 18 18 18 18 18 18 16 16  0 16 18 18 18 18 18 18 18 16
  4 14 14 14]
[15 15 14 14 14 14 14 14 14 14 14  7  7  7 15 15  7  7  1  8  1  8 18 18
 18 18 14 14 14 14 14 14 15 15 15 15 15  0 14 14 14  5 14 14 14 14 14 14
 15 15 15 15 14 14 14 14  0]
[15 15 14 14 14 14 14 14 14 14 14  7  7  7 15 15  7  7  1  8  1  8 18 18
 18 18 14 14 14 14 14 14 15 15 15 15 15  0 14 14 14  5 14 14 14 14 14 14
 15 15 15 15 14 14 14 14]
[3 6 6 6 3 3 3 6 6 3 3 3 3 3 3 3 3 3 3 3 6 6 5 3 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 3 3 6 6 6 6]
[3 6 6 6 3 3 3 6 6 3 3 3 3 3 3 3 3 3 3 3 6 6 5 3 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 3 3 6 6 6]
[ 5 16 16 16 16 16  0 16  0 16  5 16 16]
[ 5 16 16 16 16 16  0 16  0 16  5 16]
[12 12 12 12 12 12 12 12]
[12 12 12 12 12 12 12]
[ 8  2  2  2  2  1  8  1  2  2  0 16  2 10 10 10 10 16 19 19 19 19 16 16]
[ 8  2  2  2  2  1  8  1  2  2  0 16  2 10 10 10 10 16 19 19 19 19 16]
[8 1 8 2 2 2 2]
[8 

In [67]:
standardHMM_20.transmat_.argmax(axis=1)

array([ 0,  8,  2,  3,  4,  5,  6,  7,  1,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [64]:
standardHMM_20.transmat_

array([[3.39332532e-01, 2.08789059e-02, 1.16585331e-02, 5.12250055e-02,
        6.07188109e-03, 1.55988653e-01, 8.57147667e-02, 1.26734587e-02,
        1.94237773e-02, 1.97463350e-03, 9.57028311e-03, 3.88618900e-03,
        1.96472508e-02, 3.21884075e-02, 1.50671545e-02, 4.34313216e-02,
        7.86331761e-02, 1.68800076e-02, 4.65272938e-02, 2.92267698e-02],
       [1.37356803e-16, 1.39444221e-01, 2.14387861e-02, 4.48839388e-03,
        3.53134374e-03, 5.09691787e-02, 1.09908529e-01, 6.37039853e-04,
        6.37457962e-01, 1.50853275e-03, 1.55532674e-03, 6.32043064e-03,
        7.94702524e-03, 4.04339498e-21, 4.94801696e-03, 6.85596061e-11,
        4.44190881e-03, 1.37240643e-11, 4.72962749e-03, 6.73677552e-04],
       [6.24671949e-03, 2.37671854e-02, 8.55350746e-01, 5.07504426e-03,
        2.56533300e-03, 9.59486769e-03, 1.81278161e-02, 8.27021842e-03,
        4.16004488e-03, 2.86353386e-03, 1.09813393e-02, 3.77130154e-03,
        9.49902802e-03, 2.39012189e-03, 7.93442751e-03, 1.0729